In [1]:

from src.domain.model import Model
from src.domain.datamodels import DatasetConfig, ModelConfig
import torch
from src.application.handlers import (
    ModelConfigHandler,
    DatasetConfigHandler,
    TrainingConfigHandler,
    WandbConfigHandler,
    LoggingConfigHandler,
    TrainModelHandler,
)
import os
import sys

import torch
from src.application.training.phon_metrics import calculate_phoneme_wise_accuracy

from src.domain.datamodels.dataset_config import DatasetConfig
from src.domain.datamodels.encodings import BridgeEncoding
from src.domain.datamodels.model_config import ModelConfig
from src.domain.datamodels.training_config import TrainingConfig
from src.domain.dataset.bridge_dataset import BridgeDataset
from src.domain.model.model import Model

In [3]:

# Load the model checkpoint
model_config: ModelConfig = ModelConfigHandler(config_filepath='app/config/model_config.yaml').get_config()
dataset_config: DatasetConfig = DatasetConfigHandler(config_filepath='app/config/dataset_config.yaml').get_config()
training_config: TrainingConfig = TrainingConfigHandler(config_filepath='app/config/training_config.yaml').get_config()
bridge_dataset = BridgeDataset(dataset_config=dataset_config, device=training_config.device)
model=Model(
    model_config=model_config,
    dataset_config=dataset_config,
    device=training_config.device,
)
checkpoint = torch.load(training_config.checkpoint_path)
model.load_state_dict(checkpoint["model_state_dict"])

# Load the test dataset
dataset_config.dataset_filepath = 'data/tests/fry_1980.pkl'
dataset_config.max_orth_seq_len = None
dataset_config.max_phon_seq_len = None
test_dataset = BridgeDataset(dataset_config=dataset_config, device=training_config.device)

/tmp/ipykernel_52420/2153777246.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(training_config.checkpoint_path)


In [5]:
output = model.generate(test_dataset.data, pathway='p2p')

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:409: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)


In [8]:
# Prepare the true and predicted phoneme sequences
phon_pred = output.phon_vecs
phon_true = test_dataset.data.phonological.targets
phon_pred = torch.stack([torch.stack(i) for i in phon_pred])


In [9]:
phon_pred.shape, phon_true.shape

(torch.Size([299, 10, 33]), torch.Size([299, 10, 33]))

In [10]:
# Mask the padding tokens
phon_valid_mask = phon_true != 2
masked_phon_true = phon_true[phon_valid_mask]
masked_phon_pred = phon_pred[phon_valid_mask]
phoneme_wise_mask = phon_pred == phon_true

In [11]:


from src.application.training.phon_metrics import calculate_cosine_distance, calculate_euclidean_distance, calculate_phon_word_accuracy


phoneme_wise_accuracy = calculate_phoneme_wise_accuracy(
        phon_true, masked_phon_true, phoneme_wise_mask
    )
word_wise_accuracy = calculate_phon_word_accuracy(
    phon_true[:64], phoneme_wise_mask[:64]
)
cosine_accuracy = calculate_cosine_distance(phon_true, phon_pred)
euclidean_distance = calculate_euclidean_distance(phon_true, phon_pred)
phoneme_wise_accuracy, word_wise_accuracy, cosine_accuracy, euclidean_distance

(tensor(0.9992), tensor(1.), tensor(0.9999), tensor(0.0008))

In [4]:
logits = model.forward('p2p', phon_enc_input=test_dataset.data.phonological.enc_input_ids,
                               phon_enc_pad_mask=test_dataset.data.phonological.enc_pad_mask,
                               phon_dec_input=test_dataset.data.phonological.dec_input_ids,
                               phon_dec_pad_mask=test_dataset.data.phonological.dec_pad_mask)

In [ ]:
phon_pred = torch.argmax(logits["phon"], dim=1)
phon_pred.shape